In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from libs import atmos_1976 as atm
from libs import constants as cst

matlab_template = dict(
        layout=go.Layout(
                #plot layout
                width=1300,
                height=800,
                showlegend=True,
                plot_bgcolor="white",
                margin=dict(t=200), #leave some padding on top

                #title
                title_font=dict(size=24, color="black", weight='bold',family="Arial",),
                title_x=0.5,  # Center the title horizontally
                title_y=0.98,  # Position the title slightly lower vertically

                #axis
                xaxis = dict(linecolor='black', showgrid=True, showline=True, gridcolor='lightgrey', ticks='outside', tickcolor='black', tickwidth=2, ticklen=5,
                             title_font=dict(size=18, weight='bold', family='Arial'),
                             tickfont=dict(size=14, family='Arial'),
                             ),
                yaxis = dict(linecolor='black', showgrid=True, showline=True, gridcolor='lightgrey', ticks='outside', tickcolor='black', tickwidth=2, ticklen=5,
                             title_font=dict(size=18, weight='bold', family='Arial'),
                             tickfont=dict(size=14, family='Arial'),
                             ),
                yaxis2 = dict(linecolor='black', showgrid=False, showline=True, gridcolor='lightgrey', ticks='outside', tickcolor='black', tickwidth=2, ticklen=5,
                             title_font=dict(size=18, weight='bold', family='Arial'),
                             tickfont=dict(size=14, family='Arial'),
                             ),

                #legend
                legend=dict(
                        x=1.1,                # Position the legend at the far-right of the plot
                        y=1.1,                # Position the legend at the top of the plot
                        xanchor='right',    # Anchor the legend on the right side
                        yanchor='bottom',      # Anchor the legend at the top
                        bordercolor='black',# Border color around the legend (optional)
                        borderwidth=1       # Border width around the legend (optional)
                        ),
         )
)

# Level accel

In [ ]:
#Level accel and sawtooth climb
OAT=-9 #OAT in Celsius

#Level accel
LA_Time=np.array([0,9,18,28,33,40,46,56,63,71,79,87,93,101,112,120,128,136,151,164,178]) #in sec since start of level accel
LA_CAS=np.array([120,130,140,150,160,170,180,190,200,210,220,230,240,250,260,270,280,290,300,310,320]) #in knots KIAS
LA_Altitude=np.array([19580,19520,19620,19620,19620,19620,19600,19600,19600,19620,19640,19580,19560,19600,19640,19680,19680,19560,19560,19660,19620,19720])

#sawtooth climbs (use pairs from reciprocal headings)
ST_deltaT=np.array([34,19.37,42,59])
ST_deltaH=np.array([1000,1000,1000,1000])
ST_CAS=np.array([260,118]) #length must be half the length of ST_deltaT and ST_deltaH (i.e. 2 pairs of reciprocal headings)

#anchor point
low_side_anchor=0 #0 if no anchor
high_side_anchor=0 # 0 if no anchor

left_text="<b>Configuration:</b> Cruise<br><b>Power setting:</b> Maximum Continuous Thrust<br><b>Pressure altitude:</b> 19 000 ft<br><b>Weight:</b> 17,000 lb<br>"
center_text=f"<b>Data basis:</b> Flight test<br><b>Test date:</b> May 21st, 2025<br><b>Tail number:</b> N-607CF<br><b>OAT:</b> {OAT} °C<br>"


In [ ]:
#post processing anchor point
anchor=(low_side_anchor+high_side_anchor)/2

#------------------------------------------------------------------------------
#post processing sawtooth climbs
ST_Ps=np.zeros(int(len(ST_deltaT)/2))
for i in range(0,len(ST_deltaT),2):
    j=int(i/2)
    ST_Ps[j]=(ST_deltaH[i]/ST_deltaT[i]+ST_deltaH[i+1]/ST_deltaT[i+1])/2
#------------------------------------------------------------------------------
#post processing Ps
#get TAS from CAS
LA_TAS=np.zeros(len(LA_CAS))
for i in range(len(LA_CAS)):
    LA_TAS[i]=atm.CAStoTAS(LA_CAS[i],LA_Altitude[i],temp=OAT,Offset=False)

#compute Ps
LA_Ps=np.zeros(len(LA_Time))

for i in range(1,len(LA_Time)-1):
    deltaV=(LA_TAS[i+1]-LA_TAS[i-1])/2
    deltaT=(LA_Time[i+1]-LA_Time[i-1])/2
    deltaH=(LA_Altitude[i+1]-LA_Altitude[i-1])/2
    LA_Ps[i]=(LA_TAS[i]*cst.KT_TO_FPS)/cst.G_IMPERIAL*deltaV*cst.KT_TO_FPS/deltaT+deltaH/deltaT

LA_Ps[0]=LA_Ps[1]+(1/2)*(LA_Ps[1]-LA_Ps[2])
LA_Ps[-1]=LA_Ps[-2]+(1/2)*(LA_Ps[-3]-LA_Ps[-2])

if anchor==0:
    # #### Alternate second order model that does not go through the anchor point
    coeffs = np.polyfit(LA_CAS, LA_Ps, 2)

    # Create a polynomial function from the coefficients
    poly_eq = np.poly1d(coeffs)

    # Generate fitted values
    CAS_fit = np.linspace(min(LA_CAS), max(LA_CAS), 200)
    Ps_fit = poly_eq(CAS_fit)

else:
    #######################################
    #Fit second order model that must go through the anchor point

    # Subtract the constraint to reduce the problem
    # y = a*x^2 + b*x + c, and we force: y0 = a*x0^2 + b*x0 + c
    # So c = y0 - a*x0^2 - b*x0

    # Build the design matrix with reduced c
    X = np.vstack([LA_CAS**2 - anchor**2, LA_CAS - anchor]).T
    Y = LA_Ps - 0

    # Solve least squares for a and b
    [a, b], *_ = np.linalg.lstsq(X, Y, rcond=None)

    # Compute c using the constraint
    c = 0 - a*anchor**2 - b*anchor

    # Define polynomial
    def constrained_poly(x: np.ndarray) -> np.ndarray:
        """Second order polynomial."""
        return a*x**2 + b*x + c

    # Plot
    CAS_fit = np.linspace(min(LA_CAS), anchor, 500)
    Ps_fit = constrained_poly(CAS_fit)

In [ ]:
#Level accel and sawtooth climb plot
fig=go.Figure()

fig.add_trace(go.Scatter(x=LA_CAS, y=LA_Ps, mode='markers', name='Level acceleration', marker=dict(size=8, symbol='circle',color='grey')))
fig.add_trace(go.Scatter(x=ST_CAS, y=ST_Ps, mode='markers', name='Sawtooth climb',  marker=dict(size=14, symbol='triangle-up-open',color='grey')))

if anchor!=0:
    fig.add_trace(go.Scatter(x=[anchor], y=[0], mode='markers', name='Anchor point', marker=dict(size=14, symbol='star',color='grey')))

fig.add_trace(go.Scatter(x=CAS_fit, y=Ps_fit, mode='lines', name='Fitted curve', line=dict(color='black', width=2),showlegend=False,) )

fig.update_layout(
            template=matlab_template,
            title="Sabreline - Specific excess power",
            xaxis=dict(title='Calibrated airspeed, V<sub>c</sub> (KIAS)',range=[100,350],),
            yaxis=dict(title='Specific excess power, P<sub>s</sub>',range=[0,105]),
            legend=dict(font=dict(size=16, color="black",family="Arial"),
                        yanchor="bottom", y=0.9, xanchor="right", x=0.99,
                        ),
            showlegend=True,
            width=8.42*180,
            height=6.4*180,
        )

fig.add_shape(type="rect", xref="paper", yref="paper",
            x0=0, y0=1.05,     # Bottom-left corner
            x1=1, y1=1.15,     # Top-right corner
            line=dict(color="black", width=1,),
            layer="below",
        )

fig.add_annotation(x=0, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=left_text,
    showarrow=False,)

fig.add_annotation(x=0.55, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=center_text,
    showarrow=False,)

fig.show()

# Check climb

In [ ]:

# DATA for check climb

Time=np.array([0,27,54,83,113,143,164,206,234,264,302,331,380,408,453,507]) #in sec since start of check climb
Altitude= np.arange(10000, 25001, 1000) #in ft
FF_1=np.array([2100,2100,2000,2000,1900,1825,1750,1750,1700,1500,1500,1500,1500,1450,1410,1400]) #in lb/h
FF_2=np.array([2100,2200,2100,2050,2000,1875,1825,1800,1750,1700,1700,1575,1575,1520,1490,1475]) #in lb/h
Airspeed=np.array([255,255,260,260,258,260,260,260,260,260,260,260,260,260,258,258,260]) #in knots KIAS
OAT=np.array([15,13,10,9,7,5,2,2,0,-1,-3,-4,-6,-8,-11,-12]) #in Celcius

FM_time_to_climb=(11.4-3.2)*60 #in sec
FM_fuel_to_climb=(7.25-2.65)*100 #in lb
FM_distance_to_climb=61-15 #in nm

left_text="<b>Configuration:</b> Cruise<br><b>Calibrated airspeed:</b> 260 KIAS<br><b>Power setting:</b> MCT<br><b>Weight:</b> 17,650 lb<br>"
center_text="<b>Data basis:</b> Flight test<br><b>Test date:</b> May 21st, 2025<br><b>Tail number:</b> N-607CF<br><b>Temperature:</b> ISA+22°C<br>"

In [6]:
#post processing check climb
Fuel=np.zeros(len(FF_1))
Distance=np.zeros(len(FF_1))
TAS=np.zeros(len(FF_1))

TAS[0]=atm.CAStoTAS(Airspeed[0],Altitude[0],temp=OAT[0],Offset=False)

for i in range(1,len(FF_1)):
    Fuel[i]=(FF_1[i-1]+FF_2[i-1])*(Time[i]-Time[i-1])/3600+Fuel[i-1]
    TAS[i]=atm.CAStoTAS(Airspeed[i],Altitude[i],temp=OAT[i],Offset=False)
    Distance[i]=((TAS[i-1]*(Time[i]-Time[i-1])/3600)**2-((Altitude[i]-Altitude[i-1])/6076)**2)**0.5+Distance[i-1]

In [ ]:
#Plot for check climb
fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=False,  # Optional: you can customize axes sharing
        shared_yaxes=False,  # We will be using individual y-axes for each plot
        subplot_titles=['Time to climb','Fuel to climb','Distance to climb']  # Set the title for each subplot
        # Define a secondary y-axis for each subplot
    )


fig.add_trace(go.Scatter(x=Time, y=Altitude, mode='lines', name='Flight data',line=dict(color='black', width=2),),row=1, col=1)
fig.add_trace(go.Scatter(x=Fuel, y=Altitude, mode='lines', name='Flight data',line=dict(color='black', width=2),),row=2, col=1)
fig.add_trace(go.Scatter(x=Distance, y=Altitude, mode='lines', name='Flight data',line=dict(color='black', width=2),),row=3, col=1)

#Plot flight manual data
#Altitude
fig.add_trace(go.Scatter(x=[0,FM_time_to_climb], y=[Altitude[0],Altitude[-1]],name="Flight manual",mode='lines+text',line=dict(color='black', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=[0,FM_time_to_climb*0.85], y=[Altitude[0],Altitude[-1]],name='+/-15%',mode='lines',line=dict(color='black', dash='dot')), row=1, col=1)
fig.add_trace(go.Scatter(x=[0,FM_time_to_climb*1.15], y=[Altitude[0],Altitude[-1]],name='+15%', mode='lines',line=dict(color='black', dash='dot'), showlegend=False), row=1, col=1)

#Fuel
fig.add_trace(go.Scatter(x=[0,FM_fuel_to_climb], y=[Altitude[0],Altitude[-1]],name="Flight manual",mode='lines',line=dict(color='black', dash='dash'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=[0,FM_fuel_to_climb*0.85], y=[Altitude[0],Altitude[-1]],name='-15%',mode='lines',line=dict(color='black', dash='dot'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=[0,FM_fuel_to_climb*1.15], y=[Altitude[0],Altitude[-1]],name='+15%',mode='lines',line=dict(color='black', dash='dot'), showlegend=False), row=2, col=1)

#Distance
fig.add_trace(go.Scatter(x=[0,FM_distance_to_climb], y=[Altitude[0],Altitude[-1]],name="Flight manual", mode='lines',line=dict(color='black', dash='dash'), showlegend=False), row=3, col=1)
fig.add_trace(go.Scatter(x=[0,FM_distance_to_climb*0.85], y=[Altitude[0],Altitude[-1]],name='-15%', mode='lines',line=dict(color='black', dash='dot'), showlegend=False),row=3, col=1)
fig.add_trace(go.Scatter(x=[0,FM_distance_to_climb*1.15], y=[Altitude[0],Altitude[-1]],name='+15%',mode='lines',line=dict(color='black', dash='dot'), showlegend=False),row=3, col=1)

fig.update_layout(
            template=matlab_template,
            title=f"Sabreliner - Check climb {Altitude[0]:,} to {Altitude[-1]:,} ft",
            width=8.42*180,
            height=6.4*180,
            showlegend=False,
            yaxis2=dict(showgrid=True),
            xaxis_title="Time (s)",
            xaxis2_title="Fuel (lb)",
            xaxis3_title="Distance (Nm)",
            yaxis_title="Altitude (ft)",
            yaxis2_title="Altitude (ft)",
            yaxis3_title="Altitude (ft)",
            xaxis=dict(range=[0, FM_time_to_climb],),
            xaxis2=dict(range=[0, FM_fuel_to_climb],),
            xaxis3=dict(range=[0, FM_distance_to_climb],),
        )

for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=20, weight='bold')  # Set desired font size her

fig.add_shape(type="rect", xref="paper", yref="paper",
            x0=0, y0=1.05,     # Bottom-left corner
            x1=1, y1=1.15,     # Top-right corner
            line=dict(color="black", width=1,),
            layer="below",
        )

fig.add_annotation(x=0, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=left_text,
    showarrow=False,)

fig.add_annotation(x=0.55, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=center_text,
    showarrow=False,)



fig.show()

# Check descent

In [ ]:
#################################################################################
# DATA for check descent
#################################################################################

Time=np.array([0,34,63,96,129,159,201,236,270,307,344]) #in sec since start of check climb
Altitude= np.arange(18000, 7999, -1000) #in ft
FF_1=np.array([390,390,390,390,390,390,390,400,400,400,420]) #in lb/h
FF_2=np.array([350,340,340,340,340,340,360,360,380,390,400]) #in lb/h
Airspeed=np.array([200,200,200,200,200,200,199,200,200,200,200]) #in knots KIAS
OAT=np.array([-8,-6,-4,-3,-2,0,2,3,5,7,8]) #in Celcius

FM_time_to_descend=300 #in sec
FM_fuel_to_descend=65 #in lb
FM_distance_to_descend=20 #in nm

left_text="<b>Configuration:</b> Cruise<br><b>Calibrated airspeed:</b> 260 KIAS<br><b>Power setting:</b> IDLE<br><b>Weight:</b> 17,000 lb<br>"
center_text="<b>Data basis:</b> Flight test<br><b>Test date:</b> May 21st, 2025<br><b>Tail number:</b> N-607CF<br>"

In [9]:
#post processing check descent
Fuel=np.zeros(len(FF_1))
Distance=np.zeros(len(FF_1))
TAS=np.zeros(len(FF_1))

TAS[0]=atm.CAStoTAS(Airspeed[0],Altitude[0],temp=OAT[0],Offset=False)

for i in range(1,len(FF_1)):
    Fuel[i]=(FF_1[i-1]+FF_2[i-1])*(Time[i]-Time[i-1])/3600+Fuel[i-1]
    TAS[i]=atm.CAStoTAS(Airspeed[i],Altitude[i],temp=OAT[i],Offset=False)
    Distance[i]=((TAS[i-1]*(Time[i]-Time[i-1])/3600)**2-((Altitude[i]-Altitude[i-1])/6076)**2)**0.5+Distance[i-1]


In [ ]:
#plot check descent
fig = make_subplots(
        rows=3,
        cols=1,
        shared_xaxes=False,  # Optional: you can customize axes sharing
        shared_yaxes=False,  # We will be using individual y-axes for each plot
        subplot_titles=['Time to descend','Fuel to descend','Distance to descend']  # Set the title for each subplot
        # Define a secondary y-axis for each subplot
    )


fig.add_trace(go.Scatter(x=Time, y=Altitude, mode='lines', name='Flight data',line=dict(color='black', width=2),),row=1, col=1)
fig.add_trace(go.Scatter(x=Fuel, y=Altitude, mode='lines', name='Flight data',line=dict(color='black', width=2),),row=2, col=1)
fig.add_trace(go.Scatter(x=Distance, y=Altitude, mode='lines', name='Flight data',line=dict(color='black', width=2),),row=3, col=1)

#Plot flight manual data
#Altitude
fig.add_trace(go.Scatter(x=[0,FM_time_to_descend], y=[Altitude[0],Altitude[-1]],name="Flight manual",mode='lines+text',line=dict(color='black', dash='dash')), row=1, col=1)
fig.add_trace(go.Scatter(x=[0,FM_time_to_descend*0.85], y=[Altitude[0],Altitude[-1]],name='+/-15%',mode='lines',line=dict(color='black', dash='dot')), row=1, col=1)
fig.add_trace(go.Scatter(x=[0,FM_time_to_descend*1.15], y=[Altitude[0],Altitude[-1]],name='+15%', mode='lines',line=dict(color='black', dash='dot'), showlegend=False), row=1, col=1)

#Fuel
fig.add_trace(go.Scatter(x=[0,FM_fuel_to_descend], y=[Altitude[0],Altitude[-1]],name="Flight manual",mode='lines',line=dict(color='black', dash='dash'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=[0,FM_fuel_to_descend*0.85], y=[Altitude[0],Altitude[-1]],name='-15%',mode='lines',line=dict(color='black', dash='dot'), showlegend=False), row=2, col=1)
fig.add_trace(go.Scatter(x=[0,FM_fuel_to_descend*1.15], y=[Altitude[0],Altitude[-1]],name='+15%',mode='lines',line=dict(color='black', dash='dot'), showlegend=False), row=2, col=1)

#Distance
fig.add_trace(go.Scatter(x=[0,FM_distance_to_descend], y=[Altitude[0],Altitude[-1]],name="Flight manual", mode='lines',line=dict(color='black', dash='dash'), showlegend=False), row=3, col=1)
fig.add_trace(go.Scatter(x=[0,FM_distance_to_descend*0.85], y=[Altitude[0],Altitude[-1]],name='-15%', mode='lines',line=dict(color='black', dash='dot'), showlegend=False),row=3, col=1)
fig.add_trace(go.Scatter(x=[0,FM_distance_to_descend*1.15], y=[Altitude[0],Altitude[-1]],name='+15%',mode='lines',line=dict(color='black', dash='dot'), showlegend=False),row=3, col=1)

fig.update_layout(
            template=matlab_template,
            title=f"Sabreliner - Check descent {Altitude[0]:,} to {Altitude[-1]:,} ft",
            width=8.42*180,
            height=6.4*180,
            showlegend=False,
            yaxis2=dict(showgrid=True),
            xaxis_title="Time (s)",
            xaxis2_title="Fuel (lb)",
            xaxis3_title="Distance (Nm)",
            yaxis_title="Altitude (ft)",
            yaxis2_title="Altitude (ft)",
            yaxis3_title="Altitude (ft)",
        )

for annotation in fig['layout']['annotations']:
    annotation['font'] = dict(size=20, weight='bold')  # Set desired font size her

fig.add_shape(type="rect", xref="paper", yref="paper",
            x0=0, y0=1.05,     # Bottom-left corner
            x1=1, y1=1.15,     # Top-right corner
            line=dict(color="black", width=1,),
            layer="below",
        )

fig.add_annotation(x=0, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=left_text,
    showarrow=False,)

fig.add_annotation(x=0.55, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=center_text,
    showarrow=False,)

fig.show()

# Incremental drag

In [11]:
ID_deltaT=np.array([15,16]) #time to descent in sec
ID_deltaH=np.array([1000,1000]) #descent in ft
Altitude=19000 #in ft
CAS=240 #in knots KIAS
ZFW=13100 #zero fuel weight in lb
Fuel=1900 #in lb
Sref=342.1 #in ft^2
OAT=-5 #OAT in Celsius

In [ ]:
#Computations
ROD=np.average(ID_deltaH)/np.average(ID_deltaT) #rate of descent in ft/s

Vs=ROD*(OAT+cst.C_TO_K_OFFSET)/(atm.temperature(Altitude)*cst.R_TO_K) #correct for test conditions

#compute TAS from CAS
Vt=atm.CAStoTAS(CAS,Altitude,temp=OAT,Offset=False)

gamma=np.asin(Vs/Vt)    #descent angle in rad

DeltaD=(ZFW+Fuel)*np.sin(gamma)

DeltaCD=2*DeltaD/(atm.density(Altitude)*(Vt*cst.KT_TO_FPS)**2*Sref) #drag increment in ft/s

print(f'Rate of descent: {ROD*60:.4f} ft/min gamma: {np.rad2deg(gamma):.2f} deg, Vertical speed: {Vs:.2f} ft/s, Vt: {Vt:.2f} ft/s')
print(f'density: {atm.density(Altitude):.5f} slug/ft3')
print(f'Incremental drag coefficient: {DeltaCD:.4f}')

Rate of descent: 3870.9677 ft/min gamma: 12.12 deg, Vertical speed: 69.06 ft/s, Vt: 328.97 ft/s
density: 0.00131 slug/ft3
Incremental drag coefficient: 0.0456


# Level turn

In [ ]:
Bank_angle=np.array([0,30,45,26]) #in deg
LT_CAS=np.array([200,200,160,150]) #in knots KIAS
LT_TURN_RATE=np.array([0,360/146,360/100,360/74]) #in deg/s
Altitude=21000 #in ft
OAT=-13 #OAT in Celsius
Vstall=109/1.1 #Vstall at current weight level flight
Vmax=225 #Vmax with flaps out

left_text="<b>Configuration:</b> Flaps 66%<br><b>Altitude:</b> 21,000 ft<br><b>Power setting:</b> 87% N1<br><b>Weight:</b> 16,200 lb<br>"
center_text=f"<b>Data basis:</b> Flight test<br><b>Test date:</b> May 21st, 2025<br><b>Tail number:</b> N-607CF<br><b>OAT:</b> {OAT}°C<br>"

In [ ]:
#level turn post processing
LT_TAS=np.zeros(len(LT_CAS))
for i in range(len(LT_CAS)):
    LT_TAS[i]=atm.CAStoTAS(LT_CAS[i],Altitude,temp=OAT,Offset=False)

Vstall_TAS=atm.CAStoTAS(Vstall,Altitude,OAT,Offset=True)  #get the 1.1 Vstall at that altitude in TAS
Vmo=atm.CAStoTAS(Vmax,Altitude,OAT,Offset=False) #get the Vmo at that altitude in TAS
Max_g=2 #max load factor in g


fig=go.Figure()
fig.update_layout(
    template=matlab_template,
    title="Sabreliner Level turn",
    xaxis=dict(title='True Airspeed (KTAS)',range=[120,Vmo+10]),
    yaxis=dict(title='Turn rate (°/sec)',range=[0,10],),
    legend=dict(x=0.95, y=0.95,),
    width=8.42*180,
    height=6.4*180,
)

#Calculate the load factor and turn radius for the contour plot
x=np.linspace(100, Vmo, 301)
y=np.linspace(0.5, 20, 80)
TAS,TURN_RATE=np.meshgrid(x,y)
Nz=np.sqrt(((np.deg2rad(TURN_RATE)*TAS*cst.KT_TO_FPS)/cst.G_IMPERIAL)**2+1) #Load factor in g
TURN_RADIUS=(TAS*cst.KT_TO_FPS)/np.deg2rad(TURN_RATE) #Turn radius in ft

#compute the Vstall as function of the G
Vstall_alt=atm.CAStoTAS(Vstall,Altitude,OAT,Offset=True)
Vstall_maxg=Vstall_alt*np.sqrt(Max_g)
Vstall_vect=np.linspace(Vstall_alt,Vstall_maxg,150)
Omega_vect=np.rad2deg(cst.G_IMPERIAL/(Vstall_vect*cst.KT_TO_FPS)*np.sqrt((Vstall_vect/Vstall_alt)**4-1))
Omega_Vmo=np.rad2deg(cst.G_IMPERIAL/(Vmo*cst.KT_TO_FPS)*np.sqrt((Max_g)**2-1))

#plot the level turns
fig.add_trace(go.Scatter(
    x=LT_TAS,y=LT_TURN_RATE,
    name='Level turns', mode='markers',
    marker=dict(symbol='diamond-dot',size=10,color='black',),
    hovertemplate="Turn Rate: %{y:.2f}°/sec<br>" +\
                "TAS: %{x:.2f} kt<br>" + "<extra></extra>",
    showlegend=False,
    ))


#plot the turn radius and load factor contours
fig.add_trace(go.Contour(x=x, y=y, z=Nz,
    contours=dict( showlines=True, showlabels=True, start=1.5, end=2, size=1, coloring='none',),
    line=dict(color='black', width=0.5,),
    showlegend=False,
    ))

fig.add_trace(go.Contour(x=x, y=y, z=Nz,
    contours=dict( showlines=True, showlabels=True, start=2, end=2, size=1, coloring='none',labelformat="2 g Flaps extended limit"),
    line=dict(color='black', width=3),
    showlegend=False,
))

fig.add_trace(go.Contour(
    x=x, y=y, z=TURN_RADIUS,
    contours=dict( showlines=True, showlabels=True, start=2000, end=8000, size=2000, coloring='none',),
    line=dict(color='black', dash='dash'),
    showlegend=False,
))

fig.add_trace(go.Contour(
    x=x, y=y, z=TURN_RADIUS,
    contours=dict( showlines=True, showlabels=True, start=16000, end=16000, size=1000, coloring='none',),
    line=dict(color='black', dash='dash'),
    showlegend=False,
))

#plot Vstall
fig.add_trace(go.Scatter(x=Vstall_vect,y=Omega_vect,mode='lines',line=dict(color='black',width=3),showlegend=False,))

#Plot Max TAS
fig.add_trace(go.Scatter(x=[Vmo,Vmo],y=[0,Omega_Vmo],mode='lines+text',line=dict(color='black',width=3),showlegend=False,))
fig.add_annotation(x=Vmo+2, y=Omega_Vmo/2,text='<b>V<sub>FE</sub></b>',showarrow=False,font=dict(size=16,),textangle=-90)
Vstall_x=(Vstall+Vstall_maxg)/2
Vstall_y=np.rad2deg(cst.G_IMPERIAL/(Vstall_x*cst.KT_TO_FPS)*np.sqrt((Vstall_x/Vstall_alt)**4-1))+0.5
fig.add_annotation(x=Vstall_x, y=Vstall_y,text='<b>1.1 Vs</b>',showarrow=False,font=dict(size=16,),textangle=-64)

struct_limit_x=(Vmo+Vstall_maxg)*0.5
struct_limit_y=np.rad2deg(cst.G_IMPERIAL/(struct_limit_x*cst.KT_TO_FPS)*np.sqrt((Max_g)**2-1))+0.3
fig.add_annotation(x=struct_limit_x, y=struct_limit_y,text='<b>2 g Flaps extended limit</b>',showarrow=False,font=dict(size=16,),textangle=22)

fig.add_shape(type="rect", xref="paper", yref="paper",
            x0=0, y0=1.05,     # Bottom-left corner
            x1=1, y1=1.15,     # Top-right corner
            line=dict(color="black", width=1,),
            layer="below",
        )

fig.add_annotation(x=0, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=left_text,
    showarrow=False,)

fig.add_annotation(x=0.55, y=1.15, xref="paper", yref="paper",align='left',
    font=dict(size=16, color="black",family="Arial"),
    text=center_text,
    showarrow=False,)

fig.show()